In [1]:
import pandas as pd
import numpy as nm
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
data = pd.read_csv("train.csv")

In [3]:
data.head(10)

,Unnamed: 0,filename,ticker,year,preprocessed_content,ner_entities,e_score,s_score,g_score,total_score
0,0,ASX_BSX_2020.pdf,BSX,2020,style guide colour colour use imagecolour prof...,"['bk%', 'rgb', 'un', 'el ectric mine consortiu...",3.16,18.00,11.83,32.98
1,1,ASX_BSX_2022.pdf,BSX,2022,sustainability report look mining green office...,"['murray street', 'west perth', 'west perth', ...",2.83,12.86,10.32,26.02
2,2,ASX_EXR_2022.pdf,EXR,2022,report environment social governance esg basel...,"['september', 'mongolia', 'australia', 'austra...",3.81,4.28,5.86,13.94
3,3,LSE_ADM_2019.pdf,ADM,2019,corporate social responsibilty report introduc...,"['david stevens', 'csr board', 'just over yea...",16.38,14.20,5.90,36.36
4,4,LSE_ADM_2020.pdf,ADM,2020,sustainability admiral commit maintain respons...,"['year', 'health & wellbeing', 'a -month', 'on...",15.89,13.51,5.38,34.78
5,5,LSE_ADM_2021.pdf,ADM,2021,look future sustainability report customer cle...,"['annual', 'admiralgroup.co.uk', 'this year', ...",15.91,13.49,4.85,34.25
6,6,LSE_BBY_2020.pdf,BBY,2020,sustainability report guidance version content...,"['globa l sustainability', 'glossary & abbrevi...",2.11,5.20,4.60,11.92
7,7,NASDAQ_AAPL_2019.pdf,AAPL,2019,apple develop new alloy enable use percent rec...,"['apple', 'fiscal year', '•', 'lisa jackson', ...",0.55,13.58,10.47,24.32
8,8,NASDAQ_AAPL_2021.pdf,AAPL,2021,report apple report content introduction etter...,"['tim cook', 'marter chemistry', '︎', 'apple p...",0.18,7.69,8.86,16.72
9,9,NASDAQ_AAPL_2022.pdf,AAPL,2022,reportenvironmental introduction letter cook r...,"['apple', 'tim cook', 'united nations sustaina...",0.65,6.86,8.95,16.45


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            866 non-null    int64  
 1   filename              866 non-null    object 
 2   ticker                866 non-null    object 
 3   year                  866 non-null    int64  
 4   preprocessed_content  866 non-null    object 
 5   ner_entities          866 non-null    object 
 6   e_score               866 non-null    float64
 7   s_score               866 non-null    float64
 8   g_score               866 non-null    float64
 9   total_score           866 non-null    float64
dtypes: float64(4), int64(2), object(4)
memory usage: 67.8+ KB


In [5]:
data.nunique()

Unnamed: 0              866
filename                866
ticker                  263
year                     10
preprocessed_content    863
ner_entities            863
e_score                 608
s_score                 643
g_score                 537
total_score             734
dtype: int64

In [11]:
data.head()

,Unnamed: 0,preprocessed_content,e_score,s_score,g_score
0,0,style guide colour colour use imagecolour prof...,3.16,18.00,11.83
1,1,sustainability report look mining green office...,2.83,12.86,10.32
2,2,report environment social governance esg basel...,3.81,4.28,5.86
3,3,corporate social responsibilty report introduc...,16.38,14.20,5.90
4,4,sustainability admiral commit maintain respons...,15.89,13.51,5.38


In [16]:
data = data.drop(['Unnamed: 0'], axis=1)


In [17]:
data.head()

,preprocessed_content,e_score,s_score,g_score
0,style guide colour colour use imagecolour prof...,3.16,18.00,11.83
1,sustainability report look mining green office...,2.83,12.86,10.32
2,report environment social governance esg basel...,3.81,4.28,5.86
3,corporate social responsibilty report introduc...,16.38,14.20,5.90
4,sustainability admiral commit maintain respons...,15.89,13.51,5.38


In [18]:
data.to_csv('temizSirket.csv', index=False)

In [23]:
data = pd.read_csv("cleanSirket.csv")

In [31]:

data = data.dropna(subset=['preprocessed_content', 'e_score', 's_score', 'g_score'])


In [34]:
data = data[data['preprocessed_content'].apply(lambda x: isinstance(x, str))]  # Sadece string olanları seç

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Metin verisini ve 3 farklı etiketi ayıralım
text_data = data['preprocessed_content']
labels_1 = data['e_score']  # 1. kategori etiketi
labels_2 = data['s_score']  # 2. kategori etiketi
labels_3 = data['g_score']  # 3. kategori etiketi

# TF-IDF özellik çıkarımı (30000 en çok kullanılan kelime)
tfidf_vectorizer = TfidfVectorizer(max_features=30000)
X_tfidf = tfidf_vectorizer.fit_transform(text_data)

# SVD ile boyut indirgeme (400 boyut)
svd = TruncatedSVD(n_components=400, random_state=42)
X_reduced = svd.fit_transform(X_tfidf)

# Fonksiyon: Model eğitimi ve değerlendirme
def train_and_evaluate_model(X, y, label_name):
    # Eğitim ve test verilerini ayıralım
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Lineer Regresyon modeli tanımla ve eğit
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Tahmin yap
    y_pred = model.predict(X_test)
    
    # Başarım değerlendirme
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = nm.mean(nm.abs((y_test - y_pred) / y_test)) * 100

    
    print(f"\n{label_name} için Test MSE: {mse}")
    print(f"{label_name} için R² Skoru: {r2}")
    print(f"{label_name} için mape Skoru: {mape}%")

# 1. Etiket için model
train_and_evaluate_model(X_reduced, labels_1, "e_score")

# 2. Etiket için model
train_and_evaluate_model(X_reduced, labels_2, "s_score")

# 3. Etiket için model
train_and_evaluate_model(X_reduced, labels_3, "g_score")



e_score için Test MSE: 565.0575343128428
e_score için R² Skoru: -15.004598724706913
e_score için mape Skoru: 850.0609170938196%

s_score için Test MSE: 315.36110442588205
s_score için R² Skoru: -18.630280423308495
s_score için mape Skoru: 62.21181747572504%

g_score için Test MSE: 27.80616622309643
g_score için R² Skoru: -4.107147367748828
g_score için mape Skoru: 34.16846794331844%
